<a href="https://colab.research.google.com/github/Ish2276/VaR-CVaR-Model/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Download Historical Prices of the Portfolio Stocks

import yfinance as yf
import pandas as pd

# Define the ticker symbols for the portfolio
tickers = ['ASTS', 'RKLB', 'LUNR']

# Download historical prices for the portfolio
data = yf.download(tickers, start='2022-01-01', end='2024-09-18')['Adj Close']

# Calculate daily returns for each stock
returns = data.pct_change().dropna()

# Preview the data
print(returns.head())

portfolio_value = 1_000_000  # Portfolio value in dollars (1 million)

[*********************100%***********************]  3 of 3 completed

Ticker                         ASTS      LUNR      RKLB
Date                                                   
2022-01-04 00:00:00+00:00 -0.034063  0.000000 -0.045082
2022-01-05 00:00:00+00:00 -0.089421  0.000000 -0.081545
2022-01-06 00:00:00+00:00  0.024896  0.000000 -0.004673
2022-01-07 00:00:00+00:00  0.006748 -0.003096  0.019718
2022-01-10 00:00:00+00:00 -0.054960  0.000000 -0.036832


In [22]:
# Assign Portfolio Weights

from scipy.optimize import minimize

def portfolio_performance(weights, returns, risk_free_rate=0.0):
    # Calculate portfolio return
    portfolio_return = np.sum(returns.mean() * weights) * 252
    # Calculate portfolio standard deviation (risk)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    # Calculate Sharpe ratio
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

def negative_sharpe_ratio(weights, returns, risk_free_rate=0.0):
    # Minimize the negative Sharpe Ratio
    return -portfolio_performance(weights, returns, risk_free_rate)[2]

# Step 2: Set up optimization constraints and bounds
def optimize_portfolio(returns, risk_free_rate=0.0):
    num_assets = len(returns.columns)
    args = (returns, risk_free_rate)

    # Initial guess (equal weighting)
    initial_weights = num_assets * [1. / num_assets]

    # Constraints: weights must sum to 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

    # Bounds: Weights can only be between 0 and 1 (long-only portfolio)
    bounds = tuple((0, 1) for asset in range(num_assets))

    # Optimize the portfolio by minimizing the negative Sharpe Ratio
    result = minimize(negative_sharpe_ratio, initial_weights, args=args, method='SLSQP', bounds=bounds, constraints=constraints)

    return result

# Step 3: Run optimization to get weights that maximize the Sharpe Ratio
optimal_result = optimize_portfolio(returns)
optimal_weights = optimal_result.x

# Step 4: Display the optimal weights and portfolio performance
portfolio_return, portfolio_std, sharpe_ratio = portfolio_performance(optimal_weights, returns)

print(f"Optimal Weights: {optimal_weights}")
print(f"Expected Portfolio Return: {portfolio_return:.2f}")
print(f"Expected Portfolio Risk (Standard Deviation): {portfolio_std:.2f}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

Optimal Weights: [7.47869639e-01 2.52130361e-01 5.42101086e-17]
Expected Portfolio Return: 1.09
Expected Portfolio Risk (Standard Deviation): 1.04
Sharpe Ratio: 1.05


In [23]:
# Compute the covariance matrix of the daily returns

cov_matrix = returns.cov()

# Print the covariance matrix
print("Covariance Matrix:\n", cov_matrix)

Covariance Matrix:
 Ticker      ASTS      LUNR      RKLB
Ticker                              
ASTS    0.005663  0.000306  0.000985
LUNR    0.000306  0.015978  0.000034
RKLB    0.000985  0.000034  0.001811


In [24]:
# Calculate portfolio variance and standard deviation (volatility)
portfolio_variance = np.dot(optimal_weights, np.dot(cov_matrix, optimal_weights))
portfolio_volatility = np.sqrt(portfolio_variance)

print(f"Portfolio Volatility: {portfolio_volatility * 100:.2f}%")

Portfolio Volatility: 6.56%


In [32]:
# Calculate Portfolio VaR Using Variance-Covariance Method

from scipy.stats import norm

def portfolio_var(portfolio_volatility, confidence_level=0.95):
    # Use the inverse of the cumulative distribution function (ppf) of the normal distribution
    alpha = norm.ppf(1 - confidence_level)
    return portfolio_volatility * alpha

# Calculate 95% VaR using Variance-Covariance method
var_95 = portfolio_var(portfolio_volatility, 0.95)
var_95_value = var_95 * portfolio_value  # Scaling by portfolio value
print(f"95% Portfolio VaR (Variance-Covariance): ${var_95_value:.2f}")
print(f"95% Portfolio VaR (Variance-Covariance): {var_95 * 100:.2f}%")

95% Portfolio VaR (Variance-Covariance): $-107844.42
95% Portfolio VaR (Variance-Covariance): -10.78%


In [26]:
# Historical Simulation for Portfolio VaR

def historical_var_portfolio(returns, weights, confidence_level=0.95):
    # Calculate portfolio returns
    portfolio_returns = returns.dot(weights)

    # Sort the portfolio returns
    sorted_returns = portfolio_returns.sort_values()

    # Determine the VaR threshold index
    index = int((1 - confidence_level) * len(sorted_returns))

    # Return the VaR
    return sorted_returns.iloc[index]

# Calculate 95% VaR using Historical Simulation
var_historical_95 = historical_var_portfolio(returns,optimal_weights, 0.95)
print(f"95% Portfolio VaR (Historical Simulation): {var_historical_95 * 100:.2f}%")

95% Portfolio VaR (Historical Simulation): -6.97%


In [27]:
# Monte Carlo Simulation for Portfolio VaR

def monte_carlo_var_portfolio(returns, weights, num_simulations=10000, confidence_level=0.95):
    # Calculate portfolio mean and standard deviation
    portfolio_mean = np.dot(weights, returns.mean())
    portfolio_std = np.sqrt(np.dot(weights, np.dot(returns.cov(), weights)))

    # Generate random simulations
    simulated_returns = np.random.normal(portfolio_mean, portfolio_std, num_simulations)

    # Sort the simulated returns
    sorted_simulations = np.sort(simulated_returns)

    # Determine the VaR threshold index
    index = int((1 - confidence_level) * num_simulations)

    # Return the Monte Carlo VaR
    return sorted_simulations[index]

# Calculate 95% VaR using Monte Carlo Simulation
var_monte_carlo_95 = monte_carlo_var_portfolio(returns, optimal_weights, 10000, 0.95)
print(f"95% Portfolio VaR (Monte Carlo): {var_monte_carlo_95 * 100:.2f}%")

95% Portfolio VaR (Monte Carlo): -10.12%


In [28]:
# Calculate CVaR (Conditional Value at Risk)
def cvar_portfolio(returns, weights, var):
    portfolio_returns = returns.dot(weights)
    return portfolio_returns[portfolio_returns <= var].mean()

cvar_historical_95 = cvar_portfolio(returns, optimal_weights, var_historical_95)
cvar_variance_covariance_95 = cvar_portfolio(returns, optimal_weights, var_95)
cvar_monte_carlo_95 = cvar_portfolio(returns, optimal_weights, var_monte_carlo_95)

print(f"95% Portfolio CVaR (Historical Simulation): {cvar_historical_95 * 100:.2f}%")
print(f"95% Portfolio CVaR (Variance-Covariance): {cvar_variance_covariance_95 * 100:.2f}%")
print(f"95% Portfolio CVaR (Monte Carlo): {cvar_monte_carlo_95 * 100:.2f}%")

95% Portfolio CVaR (Historical Simulation): -10.40%
95% Portfolio CVaR (Variance-Covariance): -16.70%
95% Portfolio CVaR (Monte Carlo): -16.70%


In [29]:
# Backtest VaR Model
def backtest_var(returns, weights, var_estimates, confidence_level=0.95):
    portfolio_returns = returns.dot(weights)
    actual_exceedances = portfolio_returns[portfolio_returns < -var_estimates].count()
    expected_exceedances = (1 - confidence_level) * len(portfolio_returns)

    print(f"Actual Exceedances: {actual_exceedances}")
    print(f"Expected Exceedances: {expected_exceedances}")

    if actual_exceedances > expected_exceedances:
        print("Warning: The VaR model underestimates the risk.")
    else:
        print("The VaR model performs well.")

backtest_var(returns, optimal_weights, var_historical_95)

Actual Exceedances: 620
Expected Exceedances: 33.95000000000003


In [1]:
# Stress Test the Portfolio
def stress_test(returns, weights, shock_percentage=-0.1):
    stressed_returns = returns * (1 + shock_percentage)
    var_stressed_95 = historical_var_portfolio(stressed_returns, weights, 0.95)
    cvar_stressed_95 = cvar_portfolio(stressed_returns, weights, var_stressed_95)

    print(f"95% Portfolio VaR after {shock_percentage*100:.1f}% shock: {var_stressed_95 * 100:.2f}%")
    print(f"95% Portfolio CVaR after {shock_percentage*100:.1f}% shock: {cvar_stressed_95 * 100:.2f}%")

# Example: Stress test with a 10% price drop
stress_test(returns, optimal_weights, shock_percentage=-0.1)

print("hello")

NameError: name 'returns' is not defined